# 실험 09-01 증가형 MOSFET의 전압-전류 특성 실험

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import io
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def derivative(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return d

def get_simulation_result(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1
    
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def print_value_to_string(label, value):
    output = io.StringIO()
    print('%s = ' % label, end='', file=output)
    print('{:11.3f}'.format(value), file=output)
    captured = output.getvalue()
    return captured

def draw_plot(xs, ys, label, style_idx, color_idx=-1, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if color_idx < 0:
        color_idx = style_idx

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[color_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[color_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 실험 1-1 결과 분석 (실험 과정 5)

### 회로도

![Markdown-mark](../experiment/circuit_01_01_01.png)
![Markdown-mark](../experiment/circuit_01_01_02.png)

### 1. 실험 결과를 가져온다.

In [ ]:
data_l = []

data_l.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_01.csv'))
data_l.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_02.csv'))
data_l.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_03.csv'))
data_l.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_04.csv'))
data_l.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_05.csv'))
data_l.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_06.csv'))
data_l.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_07.csv'))

### 2. 실험 결과에서 $V_{DS}$, $V_{S}$를 가져온다.

In [ ]:
V_G_V_a = np.array([0.5, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0])
R_S_kOhm = 0.100

TIME_ms_a_l = []
V_D_V_a_l = []
V_S_V_a_l = []
V_DS_V_a_l = []
I_D_mA_a_l = []

for idx in range(len(V_G_V_a)):
    TIME_ms_a_l.append(data_l[idx]['TIME'] * 1000)
    V_D_V_a_l.append(data_l[idx]['Sample CH1'])
    V_S_V_a_l.append(data_l[idx]['Sample CH2'])
    V_D_V_a_l[-1] = savgol_filter(V_D_V_a_l[-1], 100, 2)
    V_S_V_a_l[-1] = savgol_filter(V_S_V_a_l[-1], 100, 2)
    V_DS_V_a_l.append(V_D_V_a_l[-1] - V_S_V_a_l[-1])
    I_D_mA_a_l.append(V_S_V_a_l[-1] / R_S_kOhm)

### 3. $V_{DS}$를 x축, $V_{S}$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('V_DS_V')
plt.ylabel('V_S_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-0.55, 5.5)
plt.ylim(-0.06, 0.6)
for idx in range(len(V_G_V_a)):
    draw_plot(V_DS_V_a_l[idx], V_S_V_a_l[idx], 'V_G_V=%3.1f)' % V_G_V_a[idx], idx)
plt.show()

In [ ]:
plt.xlabel('V_DS_V')
plt.ylabel('I_D_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-0.55, 5.5)
plt.ylim(-0.6, 6)
for idx in range(len(V_G_V_a)):
    draw_plot(V_DS_V_a_l[idx], I_D_mA_a_l[idx], 'V_G_V=%3.1f)' % V_G_V_a[idx], idx)
plt.show()

*******************************************************************************
## 실험 1-2 결과 분석 (실험 과정 6)

### 회로도

![Markdown-mark](../experiment/circuit_01_02_01.png)
![Markdown-mark](../experiment/circuit_01_02_02.png)

### 1. 실험 결과를 가져온다.

In [ ]:
I_D_mA_a_l = []

V_GG_V_a = np.array([0.5, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0])

V_DS_V_a = np.array(       [   0.1,       0.4,        0.8,         2.0,        4.0,       5.0])
I_D_mA_a_l.append(np.array([     ?,         ?,          ?,           ?,          ?,         ?])) # V_GG_V = 0.5
I_D_mA_a_l.append(np.array([     ?,         ?,          ?,           ?,          ?,         ?])) # V_GG_V = 1.0
I_D_mA_a_l.append(np.array([     ?,         ?,          ?,           ?,          ?,         ?])) # V_GG_V = 1.2
I_D_mA_a_l.append(np.array([     ?,         ?,          ?,           ?,          ?,         ?])) # V_GG_V = 1.4
I_D_mA_a_l.append(np.array([     ?,         ?,          ?,           ?,          ?,         ?])) # V_GG_V = 1.6
I_D_mA_a_l.append(np.array([     ?,         ?,          ?,           ?,          ?,         ?])) # V_GG_V = 1.8
I_D_mA_a_l.append(np.array([     ?,         ?,          ?,           ?,          ?,         ?])) # V_GG_V = 2.0

print_array('                      V_DS_V', V_DS_V_a)
for idx in range(len(V_GG_V_a)):
    print_array('V_GG_V = {:11.3f}: I_D_mA'.format(V_GG_V_a[idx]), I_D_mA_a_l[idx])

### 2. $V_{DS}$을 x축, $I_{D}$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('V_DS_V')
plt.ylabel('I_D_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-0.55, 5.5)
plt.ylim(-5.5, 55)
for idx in range(len(V_GG_V_a)):
    draw_plot(V_DS_V_a, I_D_mA_a_l[idx], '(V_GG_V={:3.1f})'.format(V_GG_V_a[idx]), idx)
plt.show()

*******************************************************************************
## 실험 2-1 결과 분석 (실험 과정 5)

### 회로도

![Markdown-mark](../experiment/circuit_01_02_01.png)
![Markdown-mark](../experiment/circuit_01_02_02.png)

### 1. 실험 결과를 가져온다.

In [ ]:
V_DS_V = 4.

V_GG_V_a = np.array([     ?,    ?,    ?,   ?,   ?,  ?,  ?,  ?])
I_D_mA_a = np.array([     ?,    ?,    ?,   ?,   ?,  ?,  ?,  ?])

print_array('V_GG_V', V_GG_V_a)
print_array('I_D_mA'.format(V_DS_V), I_D_mA_a)

### 2. $V_{GG}$을 x축, $I_{D}$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('V_GS_V')
plt.ylabel('I_D_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-0.25, 2.5)
plt.ylim(-9, 90)
draw_plot(V_GG_V_a, I_D_mA_a, 'V_DS_V=%4.3f' % V_DS_V, 0, marker_num=-1)
# plt.xlim(-0.5, 4.0)
# plt.ylim(-50, 220)
plt.show()

### 3. 문턱 전압 $V_{Tn}$을 구한다. (Second-Derivative (SD) method 사용)

In [ ]:
fx = lambda x: np.interp(x, V_GG_V_a, I_D_mA_a)

x = np.arange(V_GG_V_a[0], V_GG_V_a[-1], 0.01)
y = fx(x)
y = savgol_filter(y, 100, 2)

g = derivative(fx, x)
g = savgol_filter(g, 100, 2)

f2x = lambda x2: np.interp(x2, x, g)
g2 = derivative(f2x, x)
g2 = savgol_filter(g2, 100, 2)

g2_max = g2.max()
x_g2_max = x[g2.argmax()]

V_Tn_V = x_g2_max

plt.xlabel('V_GS_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-0.25, 2.5)
plt.ylim(-27, 270)
draw_plot(x, y, 'I_D_mA (V_DS_V=%4.3f)' % V_DS_V, 0, marker_num=0)
draw_plot(x, g, 'gm', 1, marker_num=0)
draw_plot(x, g2, 'gm2', 2, marker_num=0)
plt.scatter(x_g2_max, g2_max, color='r', zorder=10)
plt.show()

print_value("V_Tn_V", V_Tn_V)